In [1]:
import numpy as np
from utils.database_repository import DataRepository
from utils.objects import *

In [2]:
repository = DataRepository()

Connected to MySQL Server version  5.7.39-42-log


In [3]:
test_case_name = "DiningPhilosophers"
dut_name = "Surface4Pro"
dut_os = "Win32NT"
profiler_name = "IntelPowerGadget"

min_temp = 0
max_temp = 200

min_battery = 1
max_battery = 100

between = 0
duration = 2

version = 2

language = "CSharp"

In [4]:
energy_profiler = EnergyProfiler(profiler_name, repository)
dut = Dut(dut_name, dut_os, repository)
config = Configuration(min_temp, max_temp, min_battery, max_battery, duration, between, version, repository)
test_case = TestCase(test_case_name, repository)

experiment = Experiment(config.id, dut.id, test_case.id, language, repository)




(1, 'IntelPowerGadget')


In [5]:
# repository.query("SELECT * FROM RawData")